# Deploy Keras Caries-Filter Model

## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [1]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.60


In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: dl01
Azure region: uksouth
Subscription id: 51799227-bd67-4e34-96c2-fa93ef5da18d
Resource group: tom


## Find the registered model
Find the model you want to use. Prerequisite for this is that you have uploaded a registered the model on your workspace

In [4]:
from azureml.core.model import Model 
model = Model(ws, name='Caries-filter')
print(model)

Model(workspace=Workspace.create(name='dl01', subscription_id='51799227-bd67-4e34-96c2-fa93ef5da18d', resource_group='tom'), name=Caries-filter, id=Caries-filter:1, version=1, tags={}, properties={})


## Deploy the model in ACI
Now we are ready to deploy the model as a web service running in Azure Container Instance [ACI](https://azure.microsoft.com/en-us/services/container-instances/). Azure Machine Learning accomplishes this by constructing a Docker image with the scoring logic and model baked in.
### Create score.py
First, we will create a scoring script that will be invoked by the web service call. 

* Note that the scoring script must have two required functions, `init()` and `run(input_data)`. 
  * In `init()` function, you typically load the model into a global object. This function is executed only once when the Docker container is started. 
  * In `run(input_data)` function, the model is used to predict a value based on the input data. The input and output to `run` typically use JSON as serialization and de-serialization format but you are not limited to that.

In [24]:
%%writefile score.py
import json
import numpy as np
import sys
import keras
from keras.models import load_model

from azureml.core.model import Model

def init():
    global model
    print("Executing init() method...")
    print("Python version: " + str(sys.version))
    print("keras version: " + keras.__version__)

    
    # model_root = Model.get_model_path('Caries-filter')
    #model = load_model(model_root)
    print("Exiting init() method...")
    
def run(raw_data):
    print("Executing run() method...")

    #data = np.array(json.loads(raw_data)['data'])
    # make prediction
    #y_hat = np.argmax(model.predict(data), axis=1)
    y_hat = [0,1,2,3]
    return y_hat.tolist()

Overwriting score.py


### Create myenv.yml
We also need to create an environment file so that Azure Machine Learning can install the necessary packages in the Docker image which are required by your scoring script. In this case, we need to specify conda packages `tensorflow` and `keras`.

In [6]:
from azureml.core.runconfig import CondaDependencies

cd = CondaDependencies.create()
cd.add_conda_package('tensorflow')
cd.add_conda_package('keras')
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults==1.0.60.*
- tensorflow
- keras
channels:
- conda-forge



### Deploy to ACI
We are almost ready to deploy. Create a deployment configuration and specify the number of CPUs and gigbyte of RAM needed for your ACI container. 

In [18]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               auth_enabled=True, # this flag generates API keys to secure access
                                               memory_gb=1, 
                                               tags={'name':'mnist', 'framework': 'Keras'},
                                               description='Caries filter service')

#### Deployment Process
Now we can deploy. **This cell will run for about 7-8 minutes**. Behind the scene, it will do the following:
1. **Build Docker image**  
Build a Docker image using the scoring file (`score.py`), the environment file (`myenv.yml`), and the `model` object. 
2. **Register image**    
Register that image under the workspace. 
3. **Ship to ACI**    
And finally ship the image to the ACI infrastructure, start up a container in ACI using that image, and expose an HTTP endpoint to accept REST client calls.

In [25]:
from azureml.core.image import ContainerImage

imgconfig = ContainerImage.image_configuration(execution_script="score.py", 
                                               runtime="python", 
                                               conda_file="myenv.yml")

If you previously ran and it failed, you need to delete the service in the 'deployments' sectiom of the Azure Portal. You can also dete the last image if you wish since you'll be creating a new one.

In [26]:
%%time
from azureml.core.webservice import Webservice

service = Webservice.deploy_from_model(workspace=ws,
                                       name='caries-filter-service',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=imgconfig)

service.wait_for_deployment(show_output=True)

Creating image
Running...............................................
Succeeded
Image creation operation finished for image caries-filter-service:4, operation "Succeeded"
Running....................
SucceededACI service creation operation finished, operation "Succeeded"
CPU times: user 618 ms, sys: 110 ms, total: 729 ms
Wall time: 5min 44s


**Tip: If something goes wrong with the deployment, the first thing to look at is the logs from the service by running the following command:**

In [27]:
print(service.get_logs())

2019-09-18T11:47:59,537538788+00:00 - gunicorn/run 
2019-09-18T11:47:59,539910995+00:00 - iot-server/run 
2019-09-18T11:47:59,543975107+00:00 - nginx/run 
2019-09-18T11:47:59,543800607+00:00 - rsyslog/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2019-09-18T11:47:59,719790337+00:00 - iot-server/finish 1 0
2019-09-18T11:47:59,721169941+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (10)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 46
Initializing logger
Starting up app insights client
Starting up request id generator
Starting up app insight hooks
Invoking user's init function
Executing init() method...
Python version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
keras version: 2.2.5
Exiting init() method...
Users's init has completed successfully
Using TensorFlow backend.
/opt/miniconda/lib/python3.6/site-packages/tensorflow/pytho

This is the scoring web service endpoint:

In [28]:
print(service.scoring_uri)

http://71d9ac5a-97fa-4f6d-b94b-7e62b005672c.uksouth.azurecontainer.io/score


### Test the deployed model

#### Via the service object

In [ ]:
import json

# Create random image
data = np.random.randint(0, 255, size=(120, 120, 1))
test_samples = json.dumps({"data": data.tolist()})
test_samples = bytes(test_samples, encoding='utf8')


# predict using the deployed model
result = service.run(input_data=test_samples)

print(result)

### Using the URL
We can retreive the API keys used for accessing the HTTP endpoint.

In [ ]:
# retreive the API keys. two keys were generated.
key1, Key2 = service.get_keys()
print(key1)

We can now send construct raw HTTP request and send to the service. Don't forget to add key to the HTTP header.

In [ ]:
import requests

# send a random row from the test set to score
random_index = np.random.randint(0, len(X_test)-1)
input_data = "{\"data\": [" + str(list(X_test[random_index])) + "]}"

headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
#print("input data:", input_data)
print("label:", y_test[random_index])
print("prediction:", resp.text)

### Examine the Workspace
Let's look at the workspace after the web service was deployed. You should see 
* a registered model named 'keras-mlp-mnist' and with the id 'model:1'
* an image called 'keras-mnist-svc' and with a docker image location pointing to your workspace's Azure Container Registry (ACR)  
* a webservice called 'keras-mnist-svc' with some scoring URL

In [ ]:
models = ws.models
for name, model in models.items():
    print("Model: {}, ID: {}".format(name, model.id))
    
images = ws.images
for name, image in images.items():
    print("Image: {}, location: {}".format(name, image.image_location))
    
webservices = ws.webservices
for name, webservice in webservices.items():
    print("Webservice: {}, scoring URI: {}".format(name, webservice.scoring_uri))

## Clean up
You can delete the ACI deployment with a simple delete API call.

In [ ]:
service.delete()